# Fun with Ciphers
A cipher is an algorithm for encoding or decoding data. In these problems, we'll implement two very simple ciphers: the Caesar cipher and the keyword cipher. We'll use each of these to encrypt and decrypt strings.

## Caesar Cipher
A Caesar cipher is a cryptographic function that takes as input a plaintext string and outputs a ciphertext string that is the encoding of the input. Specifically, the Caesar cipher shifts each letter of the alphabet by some fixed amount, wrapping at the end of the alphabet and ignoring whitespace. For example, the string

```
CogWorks is fun
```

encoded using a Caesar cipher with shift 3 becomes

```
FrjZrunv lv ixq
```

Let's break down how this works. We'll order the alphabet so that lowercase letters come before capital letters. This gives us:

```
abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
```

The Caesar cipher will shift each letter by some amount. In our case, we chose to shift by 3 characters. This means we'll map `a` to `d`, `b` to `e`, ..., `w` to `z`, `x` to `A`, ... `W` to `Z`, `X` to `a`, `Y` to `b`, and `Z` to `c`. We can create a mapping just like this:

```
abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
defghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZabc
```

This is essentially what the Caesar cipher is doing. Now if we want to encode our string, we see that `C` maps to `F`, `o` maps to `r`, and so on.

Reversing the process to decode a string is quite straightforward. If we receive the encrypted string

```
Wlys yvjrz
```

and we know that it was encoded using a Caesar cipher with shift 7, we can easily create our reverse substitution alphabet:

```
hijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZabcdefg
abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
```

and decode the message:

```
Perl rocks
```

Fun fact: The ROT13 algorithm is a famous special case of the Caesar cipher. It assumes all letters are the same case. What's special about ROT13 is that it is its own inverse: you apply the exact same function to both encode and decode your string.

#### Breaking the Caesar Cipher
While not very secure, the Caesar cipher can be effective as a very weak form of encryption. Note that to break the encryption, we can simply try shift amounts from 1 through 25 and see which one comes out as correct English. This is a *brute-force* solution.

### Problem 1
Write a function that takes as input a string and a shift amount and returns the string encoded using a Caesar cipher with the specified shift. Your function should:

- Only replace letters; leaving numbers, punctuation, and whitespace alone
- Order the alphabet from lowercase to uppercase, as above (ab...zAB...Z)

In [7]:
def encode_caesar(string, shift_amt):
    ''' Encodes the specified `string` using a Caesar cipher with shift `shift_amt`
    
        Parameters
        ----------
        string : str
            The string to encode.
        
        shift_amt : int
            How much to shift the alphabet by.
        
        Returns
        -------
        str
            The encoded string.
       '''
    # student code goes here
    base_alphabet_string = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    caesar_string = base_alphabet_string[shift_amt:] + base_alphabet_string[0:shift_amt]
    print(caesar_string)

    output_string = str()
    for i in string:
        if i not in [letter for letter in base_alphabet_string]:
            output_string += i
        else:
            val = base_alphabet_string.find(i)
            output_string += caesar_string[base_alphabet_string.find(i)]
    
    return output_string
    

In [8]:
from bwsi_grader.python.ciphers import grade_cesar_cipher
grade_cesar_cipher(encode_caesar)

efghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZabcd
fghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZabcde
hijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZabcdefg
opqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmn
qrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnop
xyzABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvw
EFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzABCD
yzABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwx
jklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghi
efghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZabcd
JKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzABCDEFGHI
BCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzA
bcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZa
ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
zABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxy
defghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZabc
BCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzA
qrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnop
ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrst

## Keyword Cipher
A keyword cipher (or substitution cipher) is another cryptographic function. Rather than shift the alphabet by some fixed amount like the Caesar cipher, the Keyword cipher shifts the alphabet based on a keyword or phrase. Let's illustrate with an example.

Suppose our keyphrase is

```
beaverworks is cool
```

and we want to encode the string

```
python rules
```

Like the Caesar cipher, we'll build a substitution alphabet. We'll go through our keyphrase and record the **unique** letters, in order, then append all the rest of the alphabet that wasn't in the keyphrase. For simplicity, we'll only use lowercase letters for this cipher.

Going through the keyphrase we pull out

```
beavrwoksicl
```

Note that each letter is unique, and is in the order we saw it in the keyphrase. The second 'e' we see in 'beaverworks' is simply ignored. Now all we have to do is go through the alphabet and append all the letters that we don't already have. This gives us the following mapping:

```
abcdefghijklmnopqrstuvwxyz
beavrwoksicldfghjmnpqtuxyz
```

Now we can make a straightforward substitution for each letter in our string, `'python rules'`, and get the encoded string

```
hypkgf mqlrn
```

Taking another example, if we receive the encoded string

```
kqbhj khsjhp kpjige
```

and we know that the keyword used to create the cipher was

```
massachusetts institute of technology
```

then we can easily assemble the substitution alphabet

```
abcdefghijklmnopqrstuvwxyz
maschuetinoflgybdjkpqrvwxz
```

and decode the string

```
super secret string
```

#### Breaking the Keyword Cipher
You might be thinking that this is more difficult to decrypt than our Caesar cipher if we don't know the keyword, and you'd be right. With the Caesar cipher, all we had to do to break it was try 25 different shifts and see which one came out in sensible English.

With the keyword cipher, there are `403,291,461,126,605,635,584,000,000` permutations of the alphabet. And this is with only lowercase letters! Even if we could check 2 billion permutations each second, it would take more than 6 billion years to try all the possible permutations here.

Practically, this is where things like frequency analysis come into play, where if we have enough encoded text we can look at the frequency of each letter and try to match these to known frequencies of English. For example, the letter 'e' is very common and if the frequency of 'q' in the encoded text is close to that known frequency, then 'q' probably maps to 'e'. Of course, there are many sophisticated techniques in cryptanalysis worth exploring.

### Problem 2
Write a function that takes as input a string and a keyword (or phrase) and returns the string encoded using a Keyword (substitution) cipher. Your function should:

- Only replace letters; leaving numbers, punctuation, and whitespace alone
- Ignore case, using only lowercase letters

Assume that `keyword` will always be a non-empty string.

In [45]:
from collections import OrderedDict
def encode_keyword(string, keyword):
    ''' Encodes the specified `string` using a Keyword cipher with keyword `keyword`.
    
        Parameters
        ----------
        string : str
            The string to encode.
        
        keyword : str
            The keyword to use in the substitution alphabet.
        
        Returns
        -------
        str
            The encoded string.
       '''
    # student code goes here
    base_alphabet_string = 'abcdefghijklmnopqrstuvwxyz'
    #keyword = "".join(set(keyword)) # makes everything unique
    keyword = ''.join(OrderedDict.fromkeys(keyword).keys()).lower()

    cypher_string = ''.join([letter for letter in keyword if letter in base_alphabet_string])
    for letter in base_alphabet_string:
        if letter not in cypher_string:
            cypher_string += letter
        
            if len(cypher_string) == 26:
                break

    output_string = str()
    for i in string:
        if i not in [letter for letter in base_alphabet_string]:
            output_string += i
        else:
            output_string += cypher_string[base_alphabet_string.find(i)]
            print(base_alphabet_string.find(i))
            print(cypher_string)
            print(cypher_string[base_alphabet_string.find(i)])
    
    return output_string




In [46]:
from bwsi_grader.python.ciphers import grade_keyword_cipher
grade_keyword_cipher(encode_keyword)

z
t
14
ubnmdlqpjhkswrvtgacefioxyz
v
4
ubnmdlqpjhkswrvtgacefioxyz
d
11
ubnmdlqpjhkswrvtgacefioxyz
s
5
ubnmdlqpjhkswrvtgacefioxyz
l
8
ubnmdlqpjhkswrvtgacefioxyz
j
19
ubnmdlqpjhkswrvtgacefioxyz
e
14
ubnmdlqpjhkswrvtgacefioxyz
v
20
ubnmdlqpjhkswrvtgacefioxyz
f
24
ubnmdlqpjhkswrvtgacefioxyz
y
16
qfxkshzpcdolwygnabeijmrtuv
a
8
qfxkshzpcdolwygnabeijmrtuv
c
20
qfxkshzpcdolwygnabeijmrtuv
j
21
qfxkshzpcdolwygnabeijmrtuv
m
8
qfxkshzpcdolwygnabeijmrtuv
c
17
qfxkshzpcdolwygnabeijmrtuv
b
17
qfxkshzpcdolwygnabeijmrtuv
b
9
qfxkshzpcdolwygnabeijmrtuv
d
0
qfxkshzpcdolwygnabeijmrtuv
q
20
qfxkshzpcdolwygnabeijmrtuv
j
3
qfxkshzpcdolwygnabeijmrtuv
k
24
qfxkshzpcdolwygnabeijmrtuv
u
22
qfxkshzpcdolwygnabeijmrtuv
r
21
qfxkshzpcdolwygnabeijmrtuv
m
13
qfxkshzpcdolwygnabeijmrtuv
y
8
qfxkshzpcdolwygnabeijmrtuv
c
21
awhpliugyjtzkrsdmexbcfnoqv
f
12
awhpliugyjtzkrsdmexbcfnoqv
k
11
awhpliugyjtzkrsdmexbcfnoqv
z
9
awhpliugyjtzkrsdmexbcfnoqv
j
23
awhpliugyjtzkrsdmexbcfnoqv
o
5
awhpliugyjtzkrsdmexbcfnoqv
i
12
awhpliugyjtz